In [ ]:
import pandas as pd
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"]       = "1"
os.environ["MKL_NUM_THREADS"]       = "1"
os.environ["NUMEXPR_NUM_THREADS"]   = "1"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import pickle
import scanpy as sc
import anndata
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"]       = "1"
os.environ["MKL_NUM_THREADS"]       = "1"
os.environ["NUMEXPR_NUM_THREADS"]   = "1"

## Import all data

In [ ]:
# read in the lipizone - cell type correspondence data
ct=pd.read_csv("./zenodo/csv/colocalization_all_levels.csv", index_col=0)

# prepare the lipid brain atlas data, add the merfish neighbors, symmetrize
atlas = pd.read_parquet("./zenodo/maindata_2.parquet")
for i in range(2, 11):
    atlas['level_'+str(i)] = atlas['level_'+str(i-1)].astype(str) + atlas['level_'+str(i)].astype(str)
center = 456/2

# read in the merfish
merfish = pd.read_parquet("./zenodo/multimodal/cell_filtered_w500genes.parquet")
merfish = merfish.iloc[:, :-550]

# look at the nuclei density in the neighborhoods
nuclei = np.load("./zenodo/mixed/nucleidensity.npy")
sigma = 2.5
nuclei_sm = gaussian_filter(nuclei, sigma=sigma)

In [ ]:
# prepare the connectome data and its binarized version
alltests = []
for xxx in tqdm(range(30)):
    test = pd.read_parquet("./zenodo/multimodal/connectomic_datasets/connectome_batch_"+str(xxx)+".parquet")
    alltests.append(test)

connectome = pd.concat(alltests, axis=1)
connectome.index = pd.read_csv("./zenodo/csv/index_connectome.csv", index_col=0)["0"].values
connectome

In [ ]:
atlas_mid = atlas.loc[atlas['z_index'] < center,:]
coords = atlas_mid.loc[atlas_mid.index.isin(connectome.index), ['xccf', 'yccf', 'zccf']]

In [ ]:
connectome = connectome.loc[coords.index,:]
q = 0.95
cutoffs = connectome.quantile(q)
connectome_binarized = (connectome > 0) & (connectome > cutoffs)

atlas = atlas.loc[connectome.index,:]
del connectome

# prioritize fine-grained lipizones for now to start working on the results
ct=pd.read_parquet("./zenodo/csv/relaxed_colocalization_all_levels.parquet")
theyrecelltypeterritories = pd.read_csv("./zenodo/csv/theyrecelltypeterritories.csv")

ct

## Detection loop at all levels for all lipizones detected at any level as overlapping with cell types

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for iiiii in tqdm(range(ct.shape[0])):#np.random.choice(np.arange(ct.shape[0])[10:],10)):#range(ct.shape[0])[10:]):
    
    noooow = ct.iloc[iiiii,:]
    lipizonenow = ct.index.values[iiiii]
    celltypenow = noooow.putative_celltype
    levelnow = noooow.label1
    levelnow2 = noooow.label2
    atlas['putative_celltype'] = atlas[levelnow].map(ct.loc[(ct['label1'] ==levelnow) & (ct['label2'] ==levelnow2), 'putative_celltype']).fillna("nocelltype")
    atlas['xlevc'] = atlas[levelnow].map(ct.loc[(ct['label1'] ==levelnow) & (ct['label2'] ==levelnow2), 'label2']).fillna("nocelltype")

    lipizone = lipizonenow
    
    print(lipizone)
    
    bait = atlas.loc[atlas[levelnow] == lipizonenow,:]
    #bait.loc[mask, 'z_index'] = 2*center - bait.loc[mask, 'z_index']
    
    X = bait[['xccf', 'yccf', 'zccf']].values
    pca = PCA(n_components=2, random_state=0)
    X_pca = pca.fit_transform(X)
    bait['PC1'] = X_pca[:, 0]
    bait['PC2'] = X_pca[:, 1]
    kmeans = KMeans(n_clusters=2, random_state=0)
    bait['cluster'] = kmeans.fit_predict(X_pca)
    bait['clustercolor'] = "orange"
    bait.loc[bait['cluster'] ==0, 'clustercolor'] = "yellow"

    # extract the coordinates in space of that same cell type - DIRECTLY FROM THE MERFISH NOT OUR DATA, does it match one of the two spatial clusters?
    baited = merfish.loc[merfish[bait['xlevc'].unique()[0]] == bait['putative_celltype'].unique()[0],:]
    X_new = baited[['x_ccf', 'y_ccf', 'z_ccf']].values
    X_new_pca = pca.transform(X_new)
    baited['PC1'] = X_new_pca[:, 0]
    baited['PC2'] = X_new_pca[:, 1]
    baited['cluster'] = kmeans.predict(X_new_pca)
    bait['nucl'] = nuclei_sm[bait['x_index'].astype(int), bait['y_index'].astype(int), bait['z_index'].astype(int)]
    baited['nucl'] = nuclei_sm[(baited['x_ccf']*40).astype(int), (baited['y_ccf']*40).astype(int), (baited['z_ccf']*40).astype(int)]
    bait['subcell'] = "soma"

    if np.mean(baited['cluster'] == 0) > 0.8:
        bait.loc[bait['cluster'] == 1, 'subcell'] = "terminal"
    elif np.mean(baited['cluster'] == 0) < 0.2:
        bait.loc[bait['cluster'] == 0, 'subcell'] = "terminal"
    else:
        bait['subcell'] = "no_soma_terminal_detected"

    # assign soma vs nonsoma
    mediannuclei_soma = baited['nucl'].median()
    mediannuclei_terminals = bait.loc[bait['subcell'] == "terminal", 'nucl'].median()
    
    x0 = connectome_binarized.loc[connectome_binarized.index.isin(bait.loc[bait['cluster'] == 0,:].index),:]
    overlap_x0centric = x0.sum().sort_values()[::-1] / len(x0)
    candidates_x0 = overlap_x0centric[overlap_x0centric > 0.2].index.values
    x1 = connectome_binarized.loc[connectome_binarized.index.isin(bait.loc[bait['cluster'] == 1,:].index),:]
    overlap_x1centric = x1.sum().sort_values()[::-1] / len(x1)
    candidates_x1 = overlap_x1centric[overlap_x1centric > 0.2].index.values
    passthroughboth = np.intersect1d(candidates_x0, candidates_x1) 
    subdf = connectome_binarized.loc[:, passthroughboth]
    for x in subdf.columns:
        coordstream = coords.loc[subdf[x].values,:]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(coordstream.values)
        pca = PCA(n_components=1)
        t_pca = pca.fit_transform(X_scaled).ravel()
        bait[x+"_pca"] = pca.transform(bait[['xccf','yccf','zccf']]) 

    try:
        criterion = bait.loc[:, np.array(passthroughboth)+"_pca"].groupby(bait["subcell"]).mean()
        connectomestream = np.abs(criterion.loc['soma',:] - criterion.loc['terminal',:]).sort_values().index[-1][:-4]
        overlap0_val = overlap_x0centric[connectomestream]
        overlap1_val = overlap_x1centric[connectomestream]

        coordstream = coords.loc[subdf[connectomestream].values,:]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(coordstream.values)
        pca = PCA(n_components=1)
        coordstream['t_pca'] = pca.fit_transform(X_scaled).ravel()

        unique_sections = atlasss['SectionID'].unique()

        lipizonenowRN = lipizonenow.replace("/", "_")

        toperc = coordstream['t_pca'].values
        plow = np.percentile(toperc, 1)
        phigh = np.percentile(toperc, 95)

        out = {
        "bait": bait,
        "connectomestream": connectomestream,
        "mediannuclei_soma": mediannuclei_soma,
        "mediannuclei_terminals": mediannuclei_terminals, 
        "overlap_x0centric": overlap0_val,
        "overlap_x1centric": overlap1_val
        }

        with open(f"./lipizone_subcell_go/{lipizonenowRN}"+str(iiiii)+".pkl", "wb") as f:
            pickle.dump(out, f)

    except:
        continue

## Analyze the results

In [ ]:
import os
import pickle
from tqdm import tqdm

data = {}
folder = "./lipizone_subcell_go"

for fname in tqdm(os.listdir(folder)):
    if not fname.endswith(".pkl"):
        continue
    key = os.path.splitext(fname)[0]  
    path = os.path.join(folder, fname)
    with open(path, "rb") as f:
        data[key] = pickle.load(f)

ct=pd.read_parquet("./zenodo/csv/relaxed_colocalization_all_levels.parquet")
theyrecelltypeterritories = pd.read_csv("./zenodo/csv/theyrecelltypeterritories.csv")

from tqdm import tqdm

allresults = []

for NOW, NOW2 in tqdm(zip(ct.index.values, np.arange(ct.shape[0]).astype(str))):
    
    try:
        ctcorr = ct.iloc[int(NOW2),:]
        allresults.append((NOW, ctcorr['putative_celltype'],  ctcorr['colocalization'], data[NOW+NOW2]['bait']['subcell'].value_counts()["soma"]/data[NOW+NOW2]['bait'].shape[0],data[NOW+NOW2]["connectomestream"], data[NOW+NOW2]["mediannuclei_soma"] - data[NOW+NOW2]["mediannuclei_terminals"], data[NOW+NOW2]["overlap_x0centric"],data[NOW+NOW2]["overlap_x1centric"], data[NOW+NOW2]['bait']['subcell'].value_counts()["soma"], data[NOW+NOW2]['bait']['subcell'].value_counts()["terminal"])) 
    except:
        continue
        
subcellresults = pd.DataFrame(allresults, columns=['lipizone', 'celltype', 'colocalization', 'soma_fraction', 'connectomestream', 'nucleidifference', 'overlap_x0centric', 'overlap_x1centric', 'soma_count', 'terminal_count'])
subcellresults['abundance'] = subcellresults['soma_count'] + subcellresults['terminal_count']
subcellresults.to_parquet("predicted_soma_axon.parquet")

## Plot

In [ ]:
ct = ct.loc[ct['label1'] == "lipizone_names",:]
ct = (
    ct
    .sort_values('colocalization', ascending=False)
    .loc[~ct.index.duplicated(keep='first')]
)

subcellresults = subcellresults.loc[subcellresults['lipizone'].isin(ct.index),:]

ct["lc"] = ct.index + "_" + ct["putative_celltype"]
subcellresults["lc"] = subcellresults["lipizone"] + "_" + subcellresults["celltype"]
ct = ct.loc[ct["lc"].isin(subcellresults["lc"]),:]
neighs = pd.read_parquet("./zenodo/multimodal/merfishsubclassannotated_acronymized.parquet")

subcellresults

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from scipy import ndimage
from scipy.signal import savgol_filter
from shapely.geometry import Polygon
from shapely.ops import unary_union

def smooth_contours_savgol(input_array, window_length=15, polyorder=3, passes=1):
    """Extract and smooth contours using a Savitzky–Golay filter."""
    smoothed = ndimage.gaussian_filter(input_array.astype(float), sigma=0.75)
    contours = measure.find_contours(smoothed, 0.5)
    smoothed_contours = []

    for contour in contours:
        if len(contour) <= 3:
            smoothed_contours.append(contour)
            continue

        # Adjust window_length if necessary
        if len(contour) <= window_length:
            wl = min(len(contour) - 2, 5)
            if wl % 2 == 0:
                wl -= 1
        else:
            wl = window_length

        # Adjust polyorder if necessary
        po = polyorder if wl > polyorder else max(1, wl - 1)

        x, y = contour[:, 1], contour[:, 0]
        try:
            for _ in range(passes):
                x = savgol_filter(x, wl, po)
                y = savgol_filter(y, wl, po)
            smoothed_contours.append(np.column_stack((y, x)))
        except:
            smoothed_contours.append(contour)

    return smoothed_contours

ann = np.load("./zenodo/mixed/eroded_annot.npy")

def plot_gray_fill(contours, output_filename="gray_fill.pdf"):
    """Quick alternative: fill each smoothed contour in gray with a white border."""
    fig, ax = plt.subplots(figsize=(10, 8))
    for cnt in contours:
        ax.fill(cnt[:, 1], -cnt[:, 0],
                facecolor='white',
                edgecolor='lightgray',
                linewidth=6.0)

    ax.set_aspect('equal')
    ax.axis('off')
    plt.tight_layout()
    #plt.savefig(output_filename, bbox_inches='tight')
    #plt.close(fig)
    plt.show()

In [ ]:
import warnings
import numpy as np
import matplotlib.patheffects as pe

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
warnings.filterwarnings('ignore')
plt.style.use('dark_background')

for iiiii in ["SSp-bfd4-VISl4-VISp4-VISa4-AUDd4",
    "Medullary reticular nucleus, ventral part",
    "Oculomotor nucleus_1",
    "Pa4-IPDL-AV-IPI-SGN",
    "SPFm-SPA-PRC-DR-IMD",
    "VISli6a-VISl6a-AUDpo6a-VISam5-SSp-ll5","Pontine gray", "em-FF-RN-RT-III"]: 
    
    noooow = ct.loc[ct.index == iiiii,:]
    lipizonenow =noooow.index.values[0]
    celltypenow = noooow.putative_celltype[0]
    levelnow = noooow.label1[0]
    levelnow2 = noooow.label2[0]
    atlas['putative_celltype'] = atlas[levelnow].map(ct.loc[(ct['label1'] ==levelnow) & (ct['label2'] ==levelnow2), 'putative_celltype']).fillna("nocelltype")
    atlas['xlevc'] = atlas[levelnow].map(ct.loc[(ct['label1'] ==levelnow) & (ct['label2'] ==levelnow2), 'label2']).fillna("nocelltype")

    if levelnow2 == "labels_cluster":
        levelnow2 = "putative_celltype"
    nn = neighs.loc[neighs[levelnow2] == celltypenow,:]

    tmp = atlas.loc[atlas.index.isin(nn.index),:]
    
    lipizone = lipizonenow
    
    print(lipizone)
    
    bait = atlas.loc[atlas[levelnow] == lipizonenow,:]
    
    X = bait[['xccf', 'yccf', 'zccf']].values
    pca = PCA(n_components=2, random_state=0)
    X_pca = pca.fit_transform(X)
    bait['PC1'] = X_pca[:, 0]
    bait['PC2'] = X_pca[:, 1]
    kmeans = KMeans(n_clusters=2, random_state=0)
    bait['cluster'] = kmeans.fit_predict(X_pca)
    bait['clustercolor'] = "orange"
    bait.loc[bait['cluster'] ==0, 'clustercolor'] = "yellow"

    baited = merfish.loc[merfish[bait['xlevc'].unique()[0]] == bait['putative_celltype'].unique()[0],:]
    X_new = baited[['x_ccf', 'y_ccf', 'z_ccf']].values
    X_new_pca = pca.transform(X_new)
    baited['PC1'] = X_new_pca[:, 0]
    baited['PC2'] = X_new_pca[:, 1]
    baited['cluster'] = kmeans.predict(X_new_pca)
    bait['nucl'] = nuclei_sm[bait['x_index'].astype(int), bait['y_index'].astype(int), bait['z_index'].astype(int)]
    baited['nucl'] = nuclei_sm[(baited['x_ccf']*40).astype(int), (baited['y_ccf']*40).astype(int), (baited['z_ccf']*40).astype(int)]
    bait['subcell'] = "soma"

    if np.mean(baited['cluster'] == 0) > 0.8:
        bait.loc[bait['cluster'] == 1, 'subcell'] = "terminal"
    elif np.mean(baited['cluster'] == 0) < 0.2:
        bait.loc[bait['cluster'] == 0, 'subcell'] = "terminal"
    else:
        bait['subcell'] = "no_soma_terminal_detected"

    # assign soma vs nonsoma
    mediannuclei_soma = baited['nucl'].median()
    mediannuclei_terminals = bait.loc[bait['subcell'] == "terminal", 'nucl'].median()
    
    x0 = connectome_binarized.loc[connectome_binarized.index.isin(bait.loc[bait['cluster'] == 0,:].index),:]
    overlap_x0centric = x0.sum().sort_values()[::-1] / len(x0)
    candidates_x0 = overlap_x0centric[overlap_x0centric > 0.1].index.values
    x1 = connectome_binarized.loc[connectome_binarized.index.isin(bait.loc[bait['cluster'] == 1,:].index),:]
    overlap_x1centric = x1.sum().sort_values()[::-1] / len(x1)
    candidates_x1 = overlap_x1centric[overlap_x1centric > 0.1].index.values
    passthroughboth = np.intersect1d(candidates_x0, candidates_x1) 
    subdf = connectome_binarized.loc[:, passthroughboth]
    for x in subdf.columns:
        coordstream = coords.loc[subdf[x].values,:]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(coordstream.values)
        pca = PCA(n_components=1)
        t_pca = pca.fit_transform(X_scaled).ravel()
        bait[x+"_pca"] = pca.transform(bait[['xccf','yccf','zccf']])


    criterion = bait.loc[:, np.array(passthroughboth)+"_pca"].groupby(bait["subcell"]).mean() 
    connectomestream = np.abs(criterion.loc['soma',:] - criterion.loc['terminal',:]).sort_values().index[-1][:-4]

    print(f"[{lipizone}] connectomic stream → {connectomestream}")
    
    overlap0_val = overlap_x0centric[connectomestream]
    overlap1_val = overlap_x1centric[connectomestream]
    
    coordstream = coords.loc[subdf[connectomestream].values,:]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(coordstream.values)
    pca = PCA(n_components=1)
    coordstream['t_pca'] = pca.fit_transform(X_scaled).ravel()

    atlasss = atlas.loc[connectome_binarized.index,:]
    
    top32_sections = bait['SectionID'] \
        .value_counts() \
        .nlargest(32) \
        .index

    means = (
        atlasss[atlasss['SectionID'].isin(top32_sections)]
        .groupby('SectionID')['xccf']
        .mean()
    )

    unique_sections = means.sort_values(ascending=True).index.tolist()

    toperc = coordstream.loc[coordstream.index.isin(atlasss.loc[atlasss['SectionID'].isin(unique_sections),:].index), 't_pca'].values
    plow = np.percentile(toperc, 1)
    phigh = np.percentile(toperc, 95)

    soma_vals = coordstream.loc[coordstream.index.isin(bait.loc[bait['subcell'] == 'soma'].index), 't_pca']
    terminal_vals = coordstream.loc[coordstream.index.isin(bait.loc[bait['subcell'] == 'terminal'].index), 't_pca']
    
    if len(soma_vals) > 0 and len(terminal_vals) > 0:
        soma_mean = soma_vals.mean()
        terminal_mean = terminal_vals.mean()
        if soma_mean > terminal_mean:
            coordstream['t_pca'] = -coordstream['t_pca']
            plow, phigh = -phigh, -plow
    
    colors_list = ['#00C800', '#000000', '#C80000']
    custom_cmap = LinearSegmentedColormap.from_list('custom_dark_middle', colors_list, N=256)
    
    subset = atlasss.loc[atlasss['SectionID'].isin(unique_sections[:32]), ['z_index','y_index']]
    x_min, x_max = subset['z_index'].min(), subset['z_index'].max()
    y_vals = -subset['y_index']
    y_min, y_max = y_vals.min(), y_vals.max()

    
    fig, axs = plt.subplots(4, 8, figsize=(32, 16))
    fig.patch.set_facecolor('black')

    axs = axs.flatten()

    for i, section_value in enumerate(unique_sections[:32]):
        if i >= len(axs):
            break
        ax = axs[i]
        
        filtered_section = atlasss[atlasss["SectionID"] == section_value] 
        
        ahahahalol = filtered_section['x_index'].mean()
        ANN = ann[int(ahahahalol), :, :]
        contours = smooth_contours_savgol(ANN, window_length=15, polyorder=3, passes=2)
        
        midcol = ANN.shape[1] / 2

        for cnt in contours:
            left_pts = cnt[cnt[:,1] <= midcol]
            if len(left_pts) < 3:
                continue
            ax.fill(left_pts[:,1], -left_pts[:,0],
                    facecolor='none',
                    edgecolor='gray',
                    linewidth=1.5)
        
        filtered_section = atlasss[atlasss["SectionID"] == section_value] 
        filtered_section2 = coordstream.loc[coordstream.index.isin(filtered_section.index),:]
        filtered_section = filtered_section.loc[filtered_section2.index,:]
        
        ax.scatter(filtered_section['z_index'], -filtered_section['y_index'],
                        c=filtered_section2["t_pca"],s=0.2,alpha=0.75,cmap=custom_cmap, vmin=plow, vmax=phigh,
                        zorder=1, rasterized=True)   

        filtered_section = atlasss[atlasss["SectionID"] == section_value] 
        filtered_section = filtered_section.loc[filtered_section[levelnow] == lipizonenow,:]
        
        section_stream_data = coordstream.loc[coordstream.index.isin(filtered_section.index), 't_pca']

        stream_idx    = set(filtered_section2.index)

        stream_coords = atlasss.loc[list(stream_idx), ['z_index','y_index']].values

        colors = []
        for idx, row in filtered_section[['z_index','y_index']].iterrows():
            z, y = row['z_index'], row['y_index']

            if idx in stream_idx:
                keep_old = True
            else:
                deltas = np.abs(stream_coords - np.array([z, y]))
                keep_old = np.any(deltas.max(axis=1) <= 2)

            if keep_old:
                sub = bait.loc[idx, "subcell"]
                if sub == "soma":
                    colors.append("cyan")
                elif sub == "terminal":
                    colors.append("magenta")
                else:
                    colors.append("gray")
            else:
                colors.append("blue")
        
        sc = ax.scatter(
        filtered_section['z_index'], 
        -filtered_section['y_index'],
        s=12.0,             
        c=colors,     
        alpha=1.0,
        rasterized=False,   
    )

        
        sc.set_path_effects([
            pe.Stroke(linewidth=1.5, foreground='white'),
            pe.Normal()
        ])

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_aspect('equal')
        
    for ax in axs:
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_aspect('equal')
        ax.set_facecolor('black')
        ax.axis('off')

    plt.tight_layout()
    plt.savefig(
    "./lipizone_subcell_go_leaves/new_archiecturelipizones_"
    + lipizone
    + str(iiiii)
    + ".png"
    )
    plt.close()